In [1]:
# Importing lib for telegram bot
import requests
import json
import sqlite3
import time

# Debug options:
DEBUG = False

# Get Telegram bot API key from @BotFather and create base function:
BOT_API_KEY = "1644027986:AAHcEF7T2hLEJmYE3d2N7mRGOv-asZn5gwM"

# Weather API handler from weatherbit.io
WEATHER_API_KEY = "f50eb8df4f0749c5823d1b4a682ee4c5"

# Save this Update id for get new messages
update_id = 0

In [2]:
# Create sqlite3 table and get currsor
# For testing we not saved file db
if DEBUG:
    conn = sqlite3.connect(":memory:")
else:
    conn = sqlite3.connect('bot_db.sqlite')

# Getting cursor
cur = conn.cursor()


# Create tables:
def init_db():
    global conn
    global cur
    
    tables = [
        "CREATE TABLE IF NOT EXISTS users(user_id INT, name TEXT);",
        "CREATE TABLE IF NOT EXISTS user_locations(id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, user_id INT, title TEXT, lat TEXT, lon TEXT);",
        "CREATE TABLE IF NOT EXISTS jobs(id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, user_id INT, job_type TEXT, status BLOB);"
    ]
    
    for t in tables:
        cur.execute(t)
        
    conn.commit()

def get_from_db(table, user_id):
    q = f"SELECT * FROM {table} WHERE user_id = {user_id}"
    cur.execute(q)    
    return cur.fetchall()
    
def create_user(user_id, user_name):
    if not get_from_db("users", user_id):
        cur.execute("INSERT INTO users VALUES(?, ?);", (user_id, user_name))
        conn.commit()
        return True
    
def get_name(user_id):
    cur.execute(f"SELECT name FROM users WHERE user_id = {user_id};")
    return cur.fetchone()
    
def set_location(user_id, title, lan, lon):    
    cur.execute("INSERT INTO user_locations(user_id, title, lat, lon) VALUES(?, ?, ?, ?);", 
                (user_id, title, lan, lon))
    conn.commit()
    
def get_location(user_id, title=None):
    if title:
        cur.execute(f"SELECT * FROM user_locations WHERE user_id={user_id} AND title='{title}' ORDER BY id DESC;")
    else:
        cur.execute(f"SELECT * FROM user_locations WHERE user_id={user_id}")
    return cur.fetchall()

def remove_location(user_id, _id):
    cur.execute(f"DELETE FROM user_locations WHERE user_id={user_id} AND id={_id}")
    conn.commit()
    
def set_job(user_id, job_type):
    cur.execute("INSERT INTO jobs(user_id, job_type, status) VALUES(?, ?, ?);", (user_id, job_type, False))
    conn.commit()

def get_job(user_id):
    cur.execute(f"SELECT * FROM jobs WHERE user_id={user_id} AND status = False ORDER BY id DESC;")
    return cur.fetchone()

def delete_job(job_id):
    cur.execute(f"DELETE FROM jobs WHERE id = {job_id}")
    conn.commit()


# Testing functions:
if DEBUG:
    init_db()
    create_user('123', 'Jhon')
    set_location('123', 'New location', '52.33442', '34.23545')
    set_location('123', 'New location 2', '52.3321', '34.23565')

    cur.execute('SELECT * FROM users')
    print(cur.fetchall())
    print(get_location('123'))
    remove_location('123', get_location('123')[0][0])
    print(get_location('123'))

In [3]:
# Get data from lat & lon
def weather_by_location(lat, lon, days=None):
    if days:
        link = f'https://api.weatherbit.io/v2.0/forecast/daily?lat={lat}&lon={lon}&days={days}&key={WEATHER_API_KEY}'
    else:
        link = f'https://api.weatherbit.io/v2.0/current?lat={lat}&lon={lon}&key={WEATHER_API_KEY}'
    return http_handler(link)

# Get data from City, Country
#def weather_by_city(country, city, days=None):
 #   if days:
  #      link = f'https://api.weatherbit.io/v2.0/forecast/daily?city={city}&country={country}&days={days}&key={WEATHER_API_KEY}'
   # else:
    #    link = f'https://api.weatherbit.io/v2.0/current?city={city}&country={country}&key={WEATHER_API_KEY}'
    #return http_handler(link)


if DEBUG:
    # Weather forecast 6 h
    # link = f'https://api.weatherbit.io/v2.0/forecast/hourly?city={city}&country={country}&hours=6&key={WEATHER_API_KEY}'
    # Test
    # print(weather_by_city('UA', 'Kiev'))
    print(weather_by_location('50.4622066','30.4092094'))

In [16]:
idle_keyboard = [
    [
        {"text": "Weather now"},
        {"text": "Weather for 5 days"}    
    ],
    [
        {"text": "About Bot"}
    ]
]


# HTTP handler for base function:
def http_handler(link, json_data=None):
    if json_data:
        r = requests.get(link, json=json_data)
    else:
        r = requests.get(link)

    return json.loads(r.text)

# Create json for sendMessage telegram API
def create_json(chat_id, text, keyboard):
    json_data = {
        "chat_id": chat_id,
        "text": text,
        "reply_markup": 
        {
            "keyboard": keyboard,
            'one_time_keyboard': True,
            'resize_keyboard': True
        }
    }
    return json_data


# Using getUpdates method, not webhook
def getUpdates():
    link = f"https://api.telegram.org/bot{BOT_API_KEY}/getUpdates"
    json_data = {"offset": int(update_id) + 1}
    result = http_handler(link, json_data)
    
    return result['result']

def sendMessage(json_data):
    link = f"https://api.telegram.org/bot{BOT_API_KEY}/sendMessage"
    return http_handler(link, json_data)
    
def get_weather_by_location(chat_id, location):
    j = get_job(chat_id)
    if j:
        job_id, _, job_type, _ = j
    else:
        job_type = 'now'
                
    if job_type == "now":                 
        resp = weather_by_location(location['latitude'], location['longitude'])
    elif job_type == "5day":
        resp = weather_by_location(location['latitude'], location['longitude'], "5")

    if resp:
        text = ""
        print(resp)
        for d in resp['data']:
            text += f"{d['datetime'][0:10]}\n{d['temp']} ℃\n{d['weather']['description']}\n\n"
 
        if len(resp['data']) == 1:
            city_name = resp['data'][0]['city_name']
        else:
            city_name = resp['city_name']

        user_location = get_location(chat_id)
        
        if not get_location(chat_id, city_name):
            if len(user_location) < 5:
                set_location(chat_id, city_name, location['latitude'], location['longitude'])
            else:
                remove_location(chat_id, get_location(chat_id)[0][0])
                set_location(chat_id, city_name, location['latitude'], location['longitude'])
    
    if j:
        delete_job(job_id)            
    sendMessage(create_json(chat_id, text, idle_keyboard))
    
    
def check_message(result):
    global update_id

    if result:
        print(result)
        # Proccessing all message in update
        for message in result:
            
            # Save last update_id
            update_id = message['update_id']
            
            # Get message information:
            if 'message' in message.keys():
                message = message['message']
                chat_id = message['chat']['id']
            
            # Check and run command if we get
            if 'entities' in message.keys():
                if message['text'] == '/start':
                    create_user(chat_id, message['from']['first_name'])
                    # create_json(chat_id, text, keyboard)
                    ans = "Hello! I'm weather bot. What do you want?"
                    
                    sendMessage(create_json(chat_id, ans, idle_keyboard))
        
            # If we get location
            elif 'location' in message.keys():
                location = message['location']

                # (GET_WEATHER_FROM_LOCATION)
                get_weather_by_location(chat_id, location)
                
            # If we get only message:
            elif 'text' in message.keys(): 
                text = message['text']
                if text == "Weather now" or text == "Weather for 5 days":
                    
                    ans = "Chose location"
                    
                    # Check saved location and add to button + Current location + Back
                    loc = get_location(chat_id)
                    if loc:
                        loc_buttons = [
                            [
                                {"text": "Current location",
                                "request_location": True}
                            ]
                        ]
                        
                        for l in loc:
                            _, _, loc_title, _, _ = l
                            loc_buttons.append([{"text": loc_title}])

                        loc_buttons.append([{"text": "Back"}])
                            
                    else:
                        loc_buttons = [
                            [
                                {"text": "Current location",
                                "request_location": True}  
                            ],
                            [
                                {"text": "Back"}
                            ]
                        ]

                    if text == "Weather now":
                        set_job(chat_id, "now")
                    else:
                        set_job(chat_id, "5day")
                    
                    sendMessage(create_json(chat_id, ans, loc_buttons))

                elif text == "About Bot":
                    ans = "tap tap tap tap and u get weather, weeeee!"
                    sendMessage(create_json(chat_id, ans, idle_keyboard))
                    
                elif text == "Back":
                    j = get_job(chat_id)
                    if j:
                        delete_job(j[0])

                    ans = "What do you want?"
                    sendMessage(create_json(chat_id, ans, idle_keyboard))
                
                elif text:
                    location = get_location(chat_id, text)
                    
                    if location:
                        loc = {'latitude': location[0][-2], 'longitude': location[0][-1]}
                        get_weather_by_location(chat_id, loc)
                    
            
            # If we get unsupported type message:
            else:
                print('# Unsupported message #')

In [12]:
# Testing functions:
if DEBUG:
    while True:
        check_message(getUpdates())
        
        # DB:
        cur.execute('SELECT * FROM users')
        print(cur.fetchall())
        cur.execute('SELECT * FROM user_locations')
        print(cur.fetchall())
        cur.execute('SELECT * FROM jobs')
        print(cur.fetchall())
        
        time.sleep(5)

In [ ]:
init_db()
while True:
    
    check_message(getUpdates())
    time.sleep(5)

[{'update_id': 588037671, 'message': {'message_id': 766, 'from': {'id': 423376879, 'is_bot': False, 'first_name': 'Дарина', 'language_code': 'ru'}, 'chat': {'id': 423376879, 'first_name': 'Дарина', 'type': 'private'}, 'date': 1615807739, 'text': 'Weather now'}}]
[{'update_id': 588037672, 'message': {'message_id': 768, 'from': {'id': 423376879, 'is_bot': False, 'first_name': 'Дарина', 'language_code': 'ru'}, 'chat': {'id': 423376879, 'first_name': 'Дарина', 'type': 'private'}, 'date': 1615807744, 'text': 'Sofiyivska Borschagivka'}}]
{'data': [{'rh': 93, 'pod': 'd', 'lon': 30.39, 'pres': 986.1, 'timezone': 'Europe/Kiev', 'ob_time': '2021-03-15 10:30', 'country_code': 'UA', 'clouds': 100, 'ts': 1615804200, 'solar_rad': 59.6, 'state_code': '13', 'city_name': 'Sofiyivska Borschagivka', 'wind_spd': 2, 'wind_cdir_full': 'east-southeast', 'wind_cdir': 'ESE', 'slp': 1008, 'vis': 1.5, 'h_angle': 0, 'sunset': '16:03', 'dni': 839.96, 'dewpt': 4, 'snow': 0, 'uv': 1.93218, 'precip': 2.36842, 'wind_d